In [ ]:
# This code is adapted from 
# https://www.kaggle.com/sermakarevich/download-resize-clean-12-hours-44gb
# A little changes is made to satisfy our purpose

In [1]:
import logging 
import math
import os
import subprocess
from multiprocessing import Pool
from PIL import Image

In [2]:
def create_logger(filename, 
                  logger_name='logger', 
                  file_fmt='%(asctime)s %(levelname)-8s: %(message)s',
                  console_fmt='%(asctime)s | %(message)s',
                  file_level=logging.DEBUG, 
                  console_level=logging.INFO):
    
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    logger.propagate = False

    file_fmt = logging.Formatter(file_fmt)
    log_file = logging.FileHandler(filename)
    log_file.setLevel(file_level)
    log_file.setFormatter(file_fmt)
    logger.addHandler(log_file)

    console_fmt = logging.Formatter(console_fmt)
    log_console = logging.StreamHandler()
    log_console.setLevel(logging.DEBUG)
    log_console.setFormatter(console_fmt)
    logger.addHandler(log_console)

    return logger


def move_images_from_sub_to_root_folder(root_folder, subfolder):
    subfolder_content = os.listdir(subfolder)
    folders_in_subfolder = [i for i in subfolder_content if os.path.isdir(os.path.join(subfolder, i))]
    for folder_in_subfolder in folders_in_subfolder:
        subfolder_ = os.path.join(subfolder, folder_in_subfolder)
        move_images_from_sub_to_root_folder(root_folder, subfolder_)
    images = [i for i in subfolder_content if i not in folders_in_subfolder]
    for image in images:
        path_to_image = os.path.join(subfolder, image) 
        os.system(f"mv {path_to_image} ./{root_folder}/{image}")
        
        
def remove_all_subfolders_inside_folder(folder):
    folder_content = os.listdir(folder)
    subfolders = [i for i in folder_content if os.path.isdir(os.path.join(folder, i))]
    for subfolder in subfolders:
        path_to_subfolder = os.path.join(folder, subfolder)
        os.system(f'rm -r {path_to_subfolder}')
        
        
def resize_folder_images(src_dir, dst_dir, size=256):
    if not os.path.isdir(dst_dir):
        logger.info("destination directory does not exist, creating destination directory.")
        os.makedirs(dst_dir)

    image_filenames=os.listdir(src_dir)
    count = 0
    for filename in image_filenames:
        dst_filepath = os.path.join(dst_dir, filename)
        src_filepath = os.path.join(src_dir, filename)
        new_img = read_and_resize_image(src_filepath, size)
        if new_img is not None:
            new_img = new_img.convert("RGB")
            new_img.save(dst_filepath)
            count += 1
    logger.debug(f'{src_dir} files resized: {count}')
    
    
def read_and_resize_image(filepath, size):
    img = read_image(filepath)
    if img:
        img = resize_image(img, size)
    return img


def resize_image(img, size):
    if type(size) == int:
        size = (size, size)
    if len(size) > 2:
        raise ValueError("Size needs to be specified as Width, Height")
    return resize_contain(img, size)


def read_image(filepath):
    try:
        img = Image.open(filepath)
        return img
    except (OSError, Exception) as e:
        logger.debug("Can't read file {}".format(filepath))
        return None


def resize_contain(image, size, resample=Image.LANCZOS, bg_color=(255, 255, 255, 0)):
    img_format = image.format
    img = image.copy()
    img.thumbnail((size[0], size[1]), resample)
    background = Image.new('RGBA', (size[0], size[1]), bg_color)
    img_position = (
        int(math.ceil((size[0] - img.size[0]) / 2)),
        int(math.ceil((size[1] - img.size[1]) / 2))
    )
    background.paste(img, img_position)
    background.format = img_format
    return background.convert('RGB')
    
    
def download_resize_clean(index):
    try:
        if not os.path.exists('train'):
            os.system('mkdir train')

        file_index = '{0:0>3}'.format(index)
        images_file_name = f'images_{file_index}.tar'
        images_folder = images_file_name.split('.')[0]
        images_md5_file_name = f'md5.images_{file_index}.txt'
        images_tar_url = f'https://s3.amazonaws.com/google-landmark/train/{images_file_name}'
        images_md5_url = f'https://s3.amazonaws.com/google-landmark/md5sum/train/{images_md5_file_name}'

        logger.info(f'Downloading: {images_file_name} and {images_md5_file_name}')
        os.system(f'wget {images_tar_url}')
        os.system(f'wget {images_md5_url}')

        logger.debug(f'Checking file md5 and control md5')
        p = subprocess.Popen(
            ["md5sum", images_file_name], 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT
        )
        stdout, stderr = p.communicate()
        md5_images = stdout.decode("utf-8").split(' ')[0]
        md5_control = open(images_md5_file_name).read().split(' ')[0]

        if md5_images == md5_control:
            logger.debug(f'MD5 are the same: {md5_images}, {md5_control}')
            logger.debug(f'Unarchiving images into: {images_folder}')
            os.system(f'mkdir {images_folder}')
            os.system(f'tar -xf {images_file_name} -C ./{images_folder}/')

            logger.debug(f'Moving images into root folder')
            move_images_from_sub_to_root_folder(images_folder, images_folder)
            remove_all_subfolders_inside_folder(images_folder)

            logger.debug(f'Resizing images')
            resize_folder_images(
                src_dir=images_folder, 
                dst_dir='train',
                size=256
            )
            os.system(f'rm -r {images_folder}')
            os.system(f'rm {images_file_name}')
            os.system(f'rm {images_md5_file_name}')
        else:
            logger.error(f'{images_file_name} was not processed due to md5 missmatch')
    except:
        logger.error(f'FAILED TO PROCESS {images_file_name}')

In [3]:
logger = create_logger('download.log')

p = Pool(processes=10)
p.map(download_resize_clean, range(500))
p.close()

2019-11-17 00:17:17,277 | Downloading: images_013.tar and md5.images_013.txt
2019-11-17 00:17:17,277 | Downloading: images_065.tar and md5.images_065.txt
2019-11-17 00:17:17,278 | Downloading: images_000.tar and md5.images_000.txt
2019-11-17 00:17:17,277 | Downloading: images_026.tar and md5.images_026.txt
2019-11-17 00:17:17,279 | Downloading: images_052.tar and md5.images_052.txt
2019-11-17 00:17:17,279 | Downloading: images_104.tar and md5.images_104.txt
2019-11-17 00:17:17,278 | Downloading: images_039.tar and md5.images_039.txt
2019-11-17 00:17:17,278 | Downloading: images_091.tar and md5.images_091.txt
2019-11-17 00:17:17,279 | Downloading: images_078.tar and md5.images_078.txt
2019-11-17 00:17:17,280 | Downloading: images_117.tar and md5.images_117.txt
2019-11-17 00:20:22,210 | Checking file md5 and control md5
2019-11-17 00:20:22,229 | Checking file md5 and control md5
2019-11-17 00:20:23,782 | MD5 are the same: 745347146400512d9690bd1d1d4bd1f2, 745347146400512d9690bd1d1d4bd1f2

2019-11-17 00:28:15,997 | images_079 files resized: 8266
2019-11-17 00:28:16,315 | Downloading: images_080.tar and md5.images_080.txt
2019-11-17 00:28:28,419 | Checking file md5 and control md5
2019-11-17 00:28:30,225 | MD5 are the same: 1ace647d89b5caacfbbe225580b7bcf1, 1ace647d89b5caacfbbe225580b7bcf1
2019-11-17 00:28:30,226 | Unarchiving images into: images_106
2019-11-17 00:28:31,255 | Moving images into root folder
2019-11-17 00:28:42,681 | Resizing images
2019-11-17 00:29:07,069 | images_053 files resized: 8266
2019-11-17 00:29:07,320 | Downloading: images_054.tar and md5.images_054.txt
2019-11-17 00:29:18,424 | images_001 files resized: 8266
2019-11-17 00:29:18,682 | Downloading: images_002.tar and md5.images_002.txt
2019-11-17 00:29:23,811 | images_027 files resized: 8266
2019-11-17 00:29:24,063 | Downloading: images_028.tar and md5.images_028.txt
2019-11-17 00:29:39,307 | images_092 files resized: 8266
2019-11-17 00:29:39,612 | Downloading: images_093.tar and md5.images_093.tx

2019-11-17 00:39:25,690 | Unarchiving images into: images_003
2019-11-17 00:39:26,654 | Moving images into root folder
2019-11-17 00:39:40,921 | Checking file md5 and control md5
2019-11-17 00:39:41,122 | Resizing images
2019-11-17 00:39:41,279 | Checking file md5 and control md5
2019-11-17 00:39:42,649 | MD5 are the same: 768e9d465891590fb06dd0b2b9e964d0, 768e9d465891590fb06dd0b2b9e964d0
2019-11-17 00:39:42,651 | Unarchiving images into: images_081
2019-11-17 00:39:42,908 | MD5 are the same: db07d5308b4e73db0496d0cf490bde57, db07d5308b4e73db0496d0cf490bde57
2019-11-17 00:39:42,909 | Unarchiving images into: images_015
2019-11-17 00:39:43,894 | Moving images into root folder
2019-11-17 00:39:44,665 | Moving images into root folder
2019-11-17 00:39:57,680 | Resizing images
2019-11-17 00:39:58,441 | Resizing images
2019-11-17 00:40:00,492 | Checking file md5 and control md5
2019-11-17 00:40:02,549 | MD5 are the same: 4d626ecdcd838caf6a4805583d801114, 4d626ecdcd838caf6a4805583d801114
2019

2019-11-17 00:47:41,348 | Unarchiving images into: images_121
2019-11-17 00:47:42,629 | Moving images into root folder
2019-11-17 00:47:43,831 | images_030 files resized: 8266
2019-11-17 00:47:44,131 | Downloading: images_031.tar and md5.images_031.txt
2019-11-17 00:47:49,540 | Resizing images
2019-11-17 00:47:51,530 | images_109 files resized: 8266
2019-11-17 00:47:51,689 | Downloading: images_110.tar and md5.images_110.txt
2019-11-17 00:47:55,890 | Resizing images
2019-11-17 00:48:23,738 | images_082 files resized: 8266
2019-11-17 00:48:24,017 | Downloading: images_083.tar and md5.images_083.txt
2019-11-17 00:49:02,345 | Checking file md5 and control md5
2019-11-17 00:49:03,988 | MD5 are the same: f634b12fdf4c1d5ccf4aa2153653923d, f634b12fdf4c1d5ccf4aa2153653923d
2019-11-17 00:49:03,989 | Unarchiving images into: images_058
2019-11-17 00:49:04,717 | images_096 files resized: 8266
2019-11-17 00:49:05,029 | Downloading: images_097.tar and md5.images_097.txt
2019-11-17 00:49:05,110 | Mo

2019-11-17 00:57:14,846 | Unarchiving images into: images_033
2019-11-17 00:57:15,836 | Moving images into root folder
2019-11-17 00:57:33,001 | Resizing images
2019-11-17 00:58:12,446 | Checking file md5 and control md5
2019-11-17 00:58:14,066 | MD5 are the same: 5bdc9f91c8cbf737d31ad113cb1e6aa0, 5bdc9f91c8cbf737d31ad113cb1e6aa0
2019-11-17 00:58:14,068 | Unarchiving images into: images_098
2019-11-17 00:58:15,054 | Moving images into root folder
2019-11-17 00:58:31,382 | Resizing images
2019-11-17 00:59:11,744 | images_033 files resized: 8266
2019-11-17 00:59:12,079 | Downloading: images_034.tar and md5.images_034.txt
2019-11-17 00:59:25,638 | Checking file md5 and control md5
2019-11-17 00:59:27,221 | MD5 are the same: 8589a43a7c1a791227b1c9787b0d6553, 8589a43a7c1a791227b1c9787b0d6553
2019-11-17 00:59:27,223 | Unarchiving images into: images_111
2019-11-17 00:59:28,125 | Moving images into root folder
2019-11-17 00:59:29,287 | Checking file md5 and control md5
2019-11-17 00:59:30,920

...

2019-11-17 04:59:58,044 | Resizing images
2019-11-17 05:00:05,411 | Checking file md5 and control md5
2019-11-17 05:00:07,000 | MD5 are the same: f75cae760346896f7bf6570cbb542315, f75cae760346896f7bf6570cbb542315
2019-11-17 05:00:07,001 | Unarchiving images into: images_484
2019-11-17 05:00:07,875 | Moving images into root folder
2019-11-17 05:00:22,386 | Resizing images
2019-11-17 05:00:48,751 | Checking file md5 and control md5
2019-11-17 05:00:50,336 | MD5 are the same: 2df6e110cf21de5762fe2acd56163a5d, 2df6e110cf21de5762fe2acd56163a5d
2019-11-17 05:00:50,338 | Unarchiving images into: images_387
2019-11-17 05:00:51,235 | Moving images into root folder
2019-11-17 05:01:02,236 | images_421 files resized: 8265
2019-11-17 05:01:02,590 | Downloading: images_422.tar and md5.images_422.txt
2019-11-17 05:01:05,094 | Resizing images
2019-11-17 05:01:31,359 | Checking file md5 and control md5
2019-11-17 05:01:32,954 | MD5 are the same: cf21c3d5b9ff3daeb2762f489f9f3b05, cf21c3d5b9ff3daeb2762f

2019-11-17 05:11:33,456 | MD5 are the same: 308eb8ca9a94ab3868f282efdf59a8b3, 308eb8ca9a94ab3868f282efdf59a8b3
2019-11-17 05:11:33,457 | Unarchiving images into: images_410
2019-11-17 05:11:34,465 | Moving images into root folder
2019-11-17 05:11:35,250 | images_435 files resized: 8265
2019-11-17 05:11:35,601 | Downloading: images_436.tar and md5.images_436.txt
2019-11-17 05:11:48,684 | Resizing images
2019-11-17 05:11:51,798 | images_398 files resized: 8266
2019-11-17 05:11:52,127 | Downloading: images_399.tar and md5.images_399.txt
2019-11-17 05:13:00,695 | images_471 files resized: 8265
2019-11-17 05:13:00,995 | Downloading: images_472.tar and md5.images_472.txt
2019-11-17 05:13:15,018 | images_423 files resized: 8265
2019-11-17 05:13:15,309 | Downloading: images_424.tar and md5.images_424.txt
2019-11-17 05:13:38,632 | images_410 files resized: 8266
2019-11-17 05:13:38,959 | Downloading: images_411.tar and md5.images_411.txt
2019-11-17 05:14:45,265 | Checking file md5 and control md

2019-11-17 05:24:36,675 | images_460 files resized: 8265
2019-11-17 05:24:37,007 | Downloading: images_461.tar and md5.images_461.txt
2019-11-17 05:24:41,949 | Resizing images
2019-11-17 05:24:43,114 | Checking file md5 and control md5
2019-11-17 05:24:44,689 | MD5 are the same: 3728ea47ed62584c6e50afaebf698c10, 3728ea47ed62584c6e50afaebf698c10
2019-11-17 05:24:44,690 | Unarchiving images into: images_488
2019-11-17 05:24:47,710 | Moving images into root folder
2019-11-17 05:25:01,998 | Resizing images
2019-11-17 05:25:08,155 | images_472 files resized: 8265
2019-11-17 05:25:08,484 | Downloading: images_473.tar and md5.images_473.txt
2019-11-17 05:26:23,749 | images_412 files resized: 8266
2019-11-17 05:26:24,056 | Downloading: images_413.tar and md5.images_413.txt
2019-11-17 05:26:28,749 | images_498 files resized: 8265
2019-11-17 05:26:29,124 | Downloading: images_499.tar and md5.images_499.txt
2019-11-17 05:26:44,599 | images_488 files resized: 8265
2019-11-17 05:26:44,929 | Downloa

2019-11-17 05:39:25,605 | Checking file md5 and control md5
2019-11-17 05:39:27,193 | MD5 are the same: 2c46d0f0afa37df575c0bb35c97d3b0d, 2c46d0f0afa37df575c0bb35c97d3b0d
2019-11-17 05:39:27,194 | Unarchiving images into: images_451
2019-11-17 05:39:28,100 | Moving images into root folder
2019-11-17 05:39:44,893 | Resizing images
2019-11-17 05:40:15,456 | images_474 files resized: 8265
2019-11-17 05:40:15,771 | Downloading: images_475.tar and md5.images_475.txt
2019-11-17 05:41:20,551 | images_451 files resized: 8265
2019-11-17 05:41:20,868 | Downloading: images_452.tar and md5.images_452.txt
2019-11-17 05:41:35,755 | Checking file md5 and control md5
2019-11-17 05:41:37,391 | MD5 are the same: 7d48de6e7d4afe81a8300508920e8254, 7d48de6e7d4afe81a8300508920e8254
2019-11-17 05:41:37,392 | Unarchiving images into: images_426
2019-11-17 05:41:38,271 | Moving images into root folder
2019-11-17 05:41:55,386 | Resizing images
2019-11-17 05:41:58,739 | Checking file md5 and control md5
2019-11-

2019-11-17 05:57:01,559 | MD5 are the same: 7673aa018c326307f8aa084fa50c6a2f, 7673aa018c326307f8aa084fa50c6a2f
2019-11-17 05:57:01,561 | Unarchiving images into: images_466
2019-11-17 05:57:02,457 | Moving images into root folder
2019-11-17 05:57:15,681 | Checking file md5 and control md5
2019-11-17 05:57:17,268 | MD5 are the same: 3161f09f48381fe6e87a913235d73d53, 3161f09f48381fe6e87a913235d73d53
2019-11-17 05:57:17,269 | Unarchiving images into: images_477
2019-11-17 05:57:18,243 | Moving images into root folder
2019-11-17 05:57:19,716 | Resizing images
2019-11-17 05:57:35,013 | Resizing images
2019-11-17 05:58:55,249 | images_466 files resized: 8265
2019-11-17 05:58:55,542 | Downloading: images_467.tar and md5.images_467.txt
2019-11-17 05:59:09,851 | images_477 files resized: 8265
2019-11-17 05:59:10,164 | Downloading: images_478.tar and md5.images_478.txt
2019-11-17 06:01:16,580 | Checking file md5 and control md5
2019-11-17 06:01:18,167 | MD5 are the same: 90ac76ed248ccbfe7f522345